In [58]:
import datetime
import numpy as np
from collections import Counter

In [17]:
centerList= [637.40,670.16, 706.20,789.40,761.10, 802.41 ,991.26, 1074.70,1079.80	]

In [12]:
centralWave = 991.26
stepSize = 0.05
waveSteps = 18
cont = "BOTH"
numsums = 16
repeats = 4
exposureTime = 80
LCVRrelaxation = 300
modulations = 4
seconds = 1/1000
cameras =2 

now = datetime.datetime.strftime(datetime.datetime.now(),"%a %b %d %H:%M:%S %Y")
executionTime = (LCVRrelaxation+exposureTime*modulations*numsums)*waveSteps*repeats*cameras*seconds

In [13]:
print(f"{np.floor(centralWave):.0f}_{str(waveSteps).zfill(2)}wave_{stepSize}step_2beam_{numsums}sums_{repeats}reps_{cont}.rcp")
with open(f"scripts\\{np.floor(centralWave):.0f}_{str(waveSteps).zfill(2)}wave_{stepSize}step_2beam_{numsums}sums_{repeats}rep_{cont}.rcp","w") as f:
    f.write(f"DATE {now}\n")
    f.write(f"AUTHOR: rcp Generator")
    f.write(f"Expected execution time {executionTime:.1f}\n")
    f.write(f"#DATATYPE BEAM  WAVELENGTH  NUMSUMS\n")
    for repeat in range(repeats):
        for cam in ["RCAM","TCAM"]:
            for i in range(waveSteps):
                f.write(f"DATA {cam}\t{cont}\t{((i-np.floor(waveSteps/2))*stepSize)+centralWave:.3f}\t{numsums}\n")
            f.write("\n")

991_18wave_0.05step_2beam_16sums_4reps_BOTH.rcp


In [14]:
#!/usr/bin/env python3
"""UCoMP Recipe Validator"""

import os
import re
from pathlib import Path

VALID_PREFILTERS = {'637', '670', '706', '761', '789', '802', '991', '1074', '1079'}
VALID_CAMERAS = {'rcam', 'tcam'}
VALID_CONTINUUM = {'red', 'blue', 'both'}
VALID_BINARY = {'in', 'out'}
VALID_GAIN = {'high', 'low'}

BINARY_COMMANDS = {'diffuser', 'occ', 'shut', 'calib', 'cover', 'nd', 'distortiongrid', 'saveall'}

class RecipeValidator:
    def __init__(self, recipes_dir):
        self.recipes_dir = Path(recipes_dir)
        self.errors = []
        self.warnings = []
    
    def validate_all(self):
        for rcp_file in self.recipes_dir.glob('**/*.rcp'):
            self.validate_recipe(rcp_file)
        for cbk_file in self.recipes_dir.glob('**/*.cbk'):
            self.validate_cookbook(cbk_file)
        for menu_file in self.recipes_dir.glob('**/*.menu'):
            self.validate_menu(menu_file)
    
    def validate_recipe(self, filepath):
        with open(filepath) as f:
            lines = f.readlines()
        
        has_data = False
        has_exposure = False
        has_gain = False
        current_prefilter = None
        
        for i, line in enumerate(lines, 1):
            line = line.split('#')[0].strip()  # Remove comments
            if not line:
                continue
            
            parts = line.split()
            cmd = parts[0].lower()
            
            if cmd == 'data':
                has_data = True
                self._validate_data(filepath, i, parts)
                if current_prefilter:
                    self._check_wavelength_match(filepath, i, parts, current_prefilter)
            
            elif cmd == 'exposure':
                if has_data:
                    self.errors.append(f"{filepath}:{i}: EXPOSURE after DATA command")
                has_exposure = True
                self._validate_exposure(filepath, i, parts)
            
            elif cmd == 'gain':
                if has_data:
                    self.errors.append(f"{filepath}:{i}: GAIN after DATA command")
                has_gain = True
                self._validate_gain(filepath, i, parts)
            
            elif cmd == 'prefilterrange':
                self._validate_prefilterrange(filepath, i, parts)
                if len(parts) > 1:
                    current_prefilter = parts[1]
            
            elif cmd in BINARY_COMMANDS:
                self._validate_binary(filepath, i, cmd, parts)
            
            elif cmd == 'o1':
                self._validate_o1(filepath, i, parts)
            
            elif cmd == 'fw':
                self._validate_fw(filepath, i, parts)
    
    def _validate_data(self, filepath, line_num, parts):
        if len(parts) < 5:
            self.errors.append(f"{filepath}:{line_num}: DATA missing parameters")
            return
        
        camera = parts[1].lower()
        if camera not in VALID_CAMERAS:
            self.errors.append(f"{filepath}:{line_num}: Invalid camera '{parts[1]}'")
        
        continuum = parts[2].lower()
        if continuum not in VALID_CONTINUUM:
            self.errors.append(f"{filepath}:{line_num}: Invalid continuum '{parts[2]}'")
        
        try:
            wavelength = float(parts[3])
            if not (530 <= wavelength <= 1083):
                self.errors.append(f"{filepath}:{line_num}: Wavelength {wavelength} out of range")
        except ValueError:
            self.errors.append(f"{filepath}:{line_num}: Invalid wavelength '{parts[3]}'")
        
        try:
            repeats = int(parts[4])
            if not (1 <= repeats <= 16):
                self.errors.append(f"{filepath}:{line_num}: Repeats {repeats} out of range (1-16)")
        except ValueError:
            self.errors.append(f"{filepath}:{line_num}: Invalid repeats '{parts[4]}'")
    
    # ... additional validation methods ...

if __name__ == '__main__':
    validator = RecipeValidator('./Recipes')
    validator.validate_all()
    
    for error in validator.errors:
        print(f"ERROR: {error}")
    for warning in validator.warnings:
        print(f"WARNING: {warning}")

In [28]:
for fName in glob.glob('scripts/*.rcp'):
    with open(fName, 'r') as f:
        recipe_lines = f.readlines()
    non_data = 0
    data_lines = 0
    for line in recipe_lines:
        line = line.split("#")[0].strip().lower()
        if line== "" or line.startswith("author") or line.startswith("date") or line.startswith("description"):
            continue
        if line.strip().startswith("data"):
            data_lines += 1
        else:
            non_data += 1
    if non_data > 0 and data_lines >0:
        print(f"Recipe {fName} has {non_data} non-DATA lines")

Recipe scripts\dark_01wave_1beam_11sums_10rep_BOTH.rcp has 1 non-DATA lines
Recipe scripts\dark_01wave_1beam_14sums_10rep_BOTH.rcp has 1 non-DATA lines
Recipe scripts\dark_01wave_1beam_16sums_10rep_BOTH.rcp has 1 non-DATA lines


In [52]:
def get_continuum_type(red, blue, both):
    non_zero = [name for name, count in [('red', red), ('blue', blue), ('both', both)] if count > 0]
    return non_zero[0] if len(non_zero) == 1 else ('mixed' if non_zero else 'none')

In [ ]:
for fName in glob.glob('scripts/*.rcp'):
    with open(fName, 'r') as f:
        recipe_lines = f.readlines()
    has_datatype = any(line.strip().lower().startswith("#datatype") for line in recipe_lines)
    datatype = next((line for line in recipe_lines if line.strip().lower().startswith("#datatype")), "")
    description = next((line for line in recipe_lines if line.strip().lower().startswith("description")), "")
    author = next((line for line in recipe_lines if line.strip().lower().startswith("author")), "")
    date = next((line for line in recipe_lines if line.strip().lower().startswith("date")), "")
    execution_time = next((line for line in recipe_lines if line.strip().lower().startswith("#expected execution time")), "")
    #has_datatype = any(line.strip().lower().startswith("#datatype") for line in recipe_lines)
    has_data = any(line.strip().lower().startswith("data") for line in recipe_lines)
    has_tcam = sum("tcam" in line.strip().lower() for line in recipe_lines)
    has_rcam = sum("rcam" in line.strip().lower() for line in recipe_lines)
    has_blue = sum("blue" in line.strip().lower() for line in recipe_lines)
    has_red = sum("red" in line.strip().lower() for line in recipe_lines)
    has_both = sum("both" in line.strip().lower() for line in recipe_lines)
    data_lines = [line.strip() for line in recipe_lines if line.strip().lower().startswith("data")]
    sums = [int(line.strip().split()[-1]) for line in data_lines if line.strip().lower().startswith("data") and line.strip().split()[-1].isdigit()]
    repeats = list(Counter([line.strip() for line in recipe_lines if line.strip().lower().startswith("data")]).values())
    unique_repeats = len(set(repeats))
    if has_data:
        beams = 2 if has_tcam == has_rcam else 1
        cont =get_continuum_type(has_red, has_blue, has_both)
        date_text = f"DATE {datetime.datetime.now().strftime('%Y %b %d')}\n"
        if date.strip() == "":
            recipe_lines.insert(0, date_text)
        else:
            recipe_lines[recipe_lines.index(date)] = date_text
        author_text = f"AUTHOR Automated summary script\n"
        if author.strip() == "":
            recipe_lines.insert(1, author_text)
        else:
            recipe_lines[recipe_lines.index(author)] = author_text
        description_text = f"DESCRIPTION Take {len(data_lines)/beams/repeats[0]:.0f} measurements with {beams} beams {sums[0]} sums, continuum {cont} repeated {repeats[0]} times.\n\n"
        if description.strip() == "":
            recipe_lines.insert(2, description_text)
        else:
            recipe_lines[recipe_lines.index(description)] = description_text

        execution_time_text = f"#Expected execution time: {len(data_lines)*(300+(80+13.7)*sums[0]*4)/1000:.3f} seconds\n"
        if execution_time.strip() == "":
            recipe_lines.insert(3, execution_time_text)
        else:
            recipe_lines[recipe_lines.index(execution_time)] = execution_time_text
        #print(f"#Expected execution time: {len(data_lines)*(300+(80+13.7)*sums[0]*4)/1000:.3f}")
        if datatype.strip() != "":
            recipe_lines.pop(recipe_lines.index(datatype))
        with open(fName, 'w') as f:
            f.write("".join(recipe_lines))
        #print(fName)
        #print("".join(recipe_lines))
            #break
        
        #print(f"{fName} DESCRIPTION Take {len(data_lines)/beams/repeats[0]:.0f} measurements with {beams} beams {sums[0]} sums, continuum {cont} repeated {repeats[0]} times.")
        
       # break
       # print(f"{fName} DT:{has_datatype}, has both:{has_tcam == has_rcam}, CONTINUUM TYPE: {cont} {description.strip()} {repeats[0]} {unique_repeats} repeats   ")
        

In [111]:
datetime.datetime.now().strftime("%Y %b %d")#%a %b %d %H:%M:%S %Y")
#list(Counter([line.strip() for line in recipe_lines if line.strip().lower().startswith("data")]).values())

'2026 Feb 06'

In [50]:
has_blue and not has_red and not has_both
has_red and not has_blue and not has_both
has_both and not has_red and not has_blue

0

In [51]:
has_both

0